# Gamut Mapping - RGB & HSV Gamut Mapping Study Models Fitting

In [1]:
%matplotlib widget

In [2]:
from __future__ import division, unicode_literals

import colour
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.interpolate
import scipy.optimize
from functools import reduce

COLOUR_STYLE = colour.plotting.colour_style()
COLOUR_STYLE.update({
    'figure.figsize': (10.24, 5.12),
    'legend.framealpha':
    colour.plotting.COLOUR_STYLE_CONSTANTS.opacity.low
})

plt.style.use(COLOUR_STYLE)

plt.style.use('dark_background')

colour.utilities.describe_environment()

colour.utilities.filter_warnings(*[True] * 4);

*                                                                             *
*   Interpreter :                                                             *
*       python : 3.7.6 (default, Dec 30 2019, 19:38:26)                       *
*                [Clang 11.0.0 (clang-1100.0.33.16)]                          *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : v0.3.15-141-g3bebd7e9                                        *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.8.0                                                       *
*       matplotlib : 3.0.3                                                    *
*       numpy : 1.18.4                                                        *
*       scipy : 1.4.1                   

## Colour Wheel & Colour Stripe Generation

In [3]:
def colour_stripe(S=1, samples=360):
    H = np.linspace(0, 1, samples)

    HSV = colour.utilities.tstack([H, np.ones(samples) * S, np.ones(samples)])
    RGB = colour.HSV_to_RGB(HSV)
 
    return RGB[np.newaxis, ...]


def colour_wheel(samples=1024, clip_circle=True, method='Colour'):
    xx, yy = np.meshgrid(
        np.linspace(-1, 1, samples), np.linspace(-1, 1, samples))

    S = np.sqrt(xx ** 2 + yy ** 2)    
    H = (np.arctan2(xx, yy) + np.pi) / (np.pi * 2)

    HSV = colour.utilities.tstack([H, S, np.ones(H.shape)])
    RGB = colour.HSV_to_RGB(HSV)

    if clip_circle == True:
        RGB[S > 1] = 0
        A = np.where(S > 1, 0, 1)
    else:
        A = np.ones(S.shape)

    if method.lower()== 'matplotlib':
        RGB = colour.utilities.orient(RGB, '90 CW')
    elif method.lower()== 'nuke':
        RGB = colour.utilities.orient(RGB, 'Flip')
        RGB = colour.utilities.orient(RGB, '90 CW')

    R, G, B = colour.utilities.tsplit(RGB)
    
    return colour.utilities.tstack([R, G, B, A])

In [4]:
COLOUR_STRIPE = colour_stripe();

COLOUR_WHEEL = colour_wheel(360, clip_circle=False)[..., 0:3]

colour.plotting.plot_image(np.resize(COLOUR_STRIPE, [36, 360, 3]));

with plt.style.context({'figure.figsize': (10.24, 10.24)}):

    colour.plotting.plot_image(COLOUR_WHEEL);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Compression Function

In [5]:
def tanh_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + b * np.tanh((x - a) / b), x)

## Gamut Mapping Study Models Difference

In [6]:
SAMPLES = np.linspace(0, 1, 360)
THRESHOLDS = np.linspace(0, 1, 21)


def hue_offset(x, a, b, c):
    return np.sin(x * a + b) * c


def medicina_HSV_control(
        a=(np.pi * 6, np.pi, np.pi / 180),
        RGB=COLOUR_STRIPE,
        S_c=0,
        C_f=tanh_compression_function):
    H, S, V = colour.utilities.tsplit(colour.RGB_to_HSV(RGB))

    HSV = colour.utilities.tstack([(H + hue_offset(SAMPLES, *a)) % 1,
                                   C_f(S, S_c, 1 - S_c), V])

    return colour.HSV_to_RGB(HSV)


def medicina_RGB_saturation(RGB=COLOUR_STRIPE,
                            C_t=0,
                            C_f=tanh_compression_function):
    L = np.max(RGB, axis=-1)[..., np.newaxis]

    D = np.abs(RGB - L) / L

    D_c = C_f(D, C_t, 1 - C_t)

    RGB_c = L - D_c * L

    return RGB_c


RGB_S = medicina_RGB_saturation()

In [7]:
colour.plotting.plot_image(np.resize(medicina_HSV_control(), [36, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
colour.plotting.plot_image(np.resize(RGB_S, [36, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
colour.plotting.plot_image(
    np.resize(
        np.abs(medicina_HSV_control([0, 0, 0]) - RGB_S) * 5, [36, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Optimisation & Curve Fitting

After some initial tests, it seemed like the closest function to minimize the error was a triangle or sine wave: https://community.acescentral.com/t/gamut-mapping-in-cylindrical-and-conic-spaces/2870/10?u=thomas_mansencal

$\sin\bigg(x \cdot \pi \cdot 6 + \pi\bigg) * \cfrac{\pi}{\approx180}$

is almost the best fitting function for the maximum saturation case, the rounded 180 value just happens to be convenient here but the actual value is slightly lower.

In [10]:
SATURATION_SAMPLES = np.linspace(0, 6, 1000)
STRIPES = np.vstack([colour_stripe(a) for a in SATURATION_SAMPLES])

PARAMETERS = []
for threshold in THRESHOLDS:
    i = 0

    def objective_function(abc):
        stripe = STRIPES[i]
        return np.sum(
            np.abs(
                medicina_HSV_control(abc, stripe, S_c=threshold) -
                medicina_RGB_saturation(stripe, C_t=threshold)))

    parameters = []
    for i in range(len(STRIPES)):
        parameters.append(
            scipy.optimize.minimize(objective_function,
                                    (np.pi * 6, np.pi, np.pi / 180)))

    PARAMETERS.append(parameters)

In [11]:
for parameters in PARAMETERS:
    with plt.style.context({'figure.figsize': (10.24, 10.24)}):
        colour.plotting.artist()

        for parameter in parameters[1::5]:
            plt.plot(
                SAMPLES,
                np.sin(
                    np.linspace(0, 1, len(SAMPLES)) * parameter.x[0] +
                    parameter.x[1]) * parameter.x[2])

        for i in np.arange(0, 360 + 60, 60) / 360:
            plt.axvline(i, c='r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
with plt.style.context({'figure.figsize': (10.24, 10.24)}):
    colour.plotting.artist()

    Y = []
    for parameters in PARAMETERS:
        y = [parameter.x[0] for parameter in parameters]
        Y.append(y)
        plt.plot(SATURATION_SAMPLES, y)

    plt.plot(SATURATION_SAMPLES, np.mean(Y, axis=0))

with plt.style.context({'figure.figsize': (10.24, 10.24)}):
    colour.plotting.artist()

    Y = []
    for parameters in PARAMETERS:
        y = [parameter.x[1] for parameter in parameters]
        Y.append(y)
        plt.plot(SATURATION_SAMPLES, y)

    plt.plot(SATURATION_SAMPLES, np.mean(Y, axis=0))

with plt.style.context({'figure.figsize': (10.24, 10.24)}):
    colour.plotting.artist()

    Y = []
    for parameters in PARAMETERS:
        y = [parameter.x[2] for parameter in parameters]
        Y.append(y)
        plt.plot(SATURATION_SAMPLES, y)

    plt.plot(SATURATION_SAMPLES, np.mean(Y, axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
def curve_segment(x, m_lnA, m_B, m_offsetX, m_offsetY, m_scaleX=1, m_scaleY=1):
    x0 = (x - m_offsetX) * m_scaleX
    y0 = np.exp(m_lnA + m_B * np.log(x0))

    return np.where(x > 0, y0 * m_scaleY + m_offsetY, 0)


def solve_A_B(x0, y0, m):
    B = (m * x0) / y0
    lnA = np.log(y0) - B * np.log(x0)

    return lnA, B


def as_slope_intercept(x0, x1, y0, y1):
    dx = (x1 - x0)
    dy = (y1 - y0)

    m = 1 if dx == 0 else dy / dx
    b = y0 - x0 * m

    return m, b


def eval_derivative_linear_gamma(m, b, g, x):
    return g * m * np.power(m * x + b, g - 1.0)


def curve(x, g, m_x0, m_y0, m_x1, m_y1, m_overshoot_x, m_overshoot_y):
    m_x1 = max(m_x0, m_x1)
    m_y1 = max(m_y0, m_y1)

    m_overshoot_x = max(m_x1 - 1, m_overshoot_x)
    m_overshoot_y = max(m_y1 - 1, m_overshoot_y)

    m, b = as_slope_intercept(m_x0, m_x1, m_y0, m_y1)

    linear_segment = curve_segment(x, g * np.log(m), g, -(b / m), 0)

    toe_M = eval_derivative_linear_gamma(m, b, g, m_x0)
    shoulder_M = eval_derivative_linear_gamma(m, b, g, m_x1)

    m_y0 = np.power(m_y0, g)
    m_y1 = np.power(m_y1, g)
    m_overshoot_y = np.power(1 + m_overshoot_y, g) - 1

    A_t, B_t = solve_A_B(m_x0, m_y0, toe_M)
    toe_segment = curve_segment(x, A_t, B_t, 0, 0)

    x0 = (1 + m_overshoot_x) - m_x1
    y0 = (1 + m_overshoot_y) - m_y1
    A_s, B_s = solve_A_B(x0, y0, shoulder_M)
    shoulder_segment = curve_segment(x, A_s, B_s, 1 + m_overshoot_x,
                                     1 + m_overshoot_y, -1, -1)

    y = np.select(
        [
            x < m_x0,
            np.logical_and(x > m_x0, x < m_x1),
            np.logical_and(x > m_x1, x < m_overshoot_x + 1),
            x > m_overshoot_x + 1,
        ],
        [
            toe_segment, linear_segment, shoulder_segment,
            np.nanmax(shoulder_segment, )
        ],
    )

    return y


partial_curve = lambda x, m_x0, m_y0, m_x1, m_y1, m_overshoot_x: curve(
    x, 1.25, m_x0, m_y0, m_x1, m_y1, m_overshoot_x, -0.86)

In [14]:
POPTS = []
for parameters in PARAMETERS:
    offset_r = [parameter.x[2] for parameter in parameters]

    offset_r[0] = 0

    popt, pcov = scipy.optimize.curve_fit(
        partial_curve,
        SATURATION_SAMPLES,
        offset_r,
        bounds=np.transpose([
            (0.4, 1),
            (0.005, 0.01),
            (1.25, 1.55),
            (0.04, 0.07),
            (1, 8),
        ]),
        maxfev=50000)
    
    POPTS.append(popt)
    print(popt)
    print(
        colour.utilities.metric_mse(offset_r, partial_curve(SATURATION_SAMPLES,
                                                    *popt)))

    with plt.style.context({'figure.figsize': (10.24, 10.24)}):
        colour.plotting.artist()

        plt.plot(SATURATION_SAMPLES, offset_r)
        plt.plot(SATURATION_SAMPLES, partial_curve(SATURATION_SAMPLES, *popt), 'o')

[ 0.42061353  0.01        1.54999165  0.06898502  8.        ]
1.42190622496e-07


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.44945874  0.01        1.5458702   0.06998316  8.        ]
1.09070593989e-07


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.48519176  0.01        1.52243267  0.07        8.        ]
1.09009430496e-07


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.98948904  0.005       1.25        0.04603146  3.67331023]
7.32129649302e-07


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.99201562  0.005       1.25        0.04588997  3.72672641]
7.46385075602e-07


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
print(np.transpose(POPTS))

with plt.style.context({'figure.figsize': (10.24, 10.24)}):
    colour.plotting.artist()
    
    for popt in POPTS:
        plt.plot(SATURATION_SAMPLES, partial_curve(SATURATION_SAMPLES, *popt))
        

for popt in np.transpose(POPTS):
    with plt.style.context({'figure.figsize': (10.24, 10.24)}):
        colour.plotting.artist()
        
        plt.plot(popt)

[[  4.20613529e-01   4.49458739e-01   4.85191758e-01   5.23669259e-01
    5.64343322e-01   6.05376678e-01   6.46719612e-01   6.87411661e-01
    7.24820949e-01   7.62348310e-01   8.02763652e-01   8.41166592e-01
    8.75980594e-01   9.07221789e-01   9.35055515e-01   9.59761498e-01
    9.81849512e-01   9.89749348e-01   9.82020536e-01   9.89489039e-01
    9.92015624e-01]
 [  9.99999990e-03   9.99999980e-03   1.00000000e-02   1.00000000e-02
    9.99995100e-03   9.99999975e-03   9.99999976e-03   1.00000000e-02
    1.00000000e-02   1.00000000e-02   1.00000000e-02   1.00000000e-02
    1.00000000e-02   1.00000000e-02   1.00000000e-02   1.00000000e-02
    1.00000000e-02   8.42324260e-03   5.00000000e-03   5.00000000e-03
    5.00000000e-03]
 [  1.54999165e+00   1.54587020e+00   1.52243267e+00   1.50167967e+00
    1.48373347e+00   1.46864029e+00   1.45617162e+00   1.44643813e+00
    1.43992726e+00   1.43489649e+00   1.40494720e+00   1.37700835e+00
    1.35949114e+00   1.34585395e+00   1.33495790e+

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
with plt.style.context({'figure.figsize': (10.24, 10.24)}):
    colour.plotting.artist()
    
    plt.plot(SATURATION_SAMPLES, partial_curve(SATURATION_SAMPLES, *POPTS[0]))
    plt.plot(SATURATION_SAMPLES, partial_curve(SATURATION_SAMPLES, *POPTS[len(POPTS) // 2]))    
    plt.plot(SATURATION_SAMPLES, partial_curve(SATURATION_SAMPLES, *POPTS[-1]))
    plt.plot(SATURATION_SAMPLES, partial_curve(SATURATION_SAMPLES, *((POPTS[0] + POPTS[-1]) / 2)), '--')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
def hue_offset_extended(x, S, m_x0, m_y0, m_x1, m_y1, m_overshoot_x):
    return np.sin(x * np.pi * 6 + np.pi) * partial_curve(S, m_x0, m_y0, m_x1, m_y1, m_overshoot_x)


def medicina_HSV_control_extended(a=POPTS[0],
                                  RGB=COLOUR_STRIPE,
                                  S_c=0,
                                  C_f=tanh_compression_function):
    H, S, V = colour.utilities.tsplit(colour.RGB_to_HSV(RGB))

    HSV = colour.utilities.tstack(
        [(H + hue_offset_extended(H, S, *a)) % 1,
          C_f(S, S_c, 1 - S_c), V])

    return colour.HSV_to_RGB(HSV)


with plt.style.context({'figure.figsize': (10.24, 10.24)}):
    # colour.plotting.plot_image(STRIPES);

    colour.plotting.plot_image(
            np.abs(
                medicina_HSV_control([0, 0, 0], RGB=STRIPES) -
                medicina_RGB_saturation(STRIPES)) * 5);

    colour.plotting.plot_image(
            np.abs(
                medicina_HSV_control_extended(RGB=STRIPES) -
                medicina_RGB_saturation(STRIPES)) * 5);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
with plt.style.context({'figure.figsize': (10.24, 10.24)}):
    colour.plotting.plot_image(
        np.abs(
            medicina_HSV_control([0, 0, 0], RGB=COLOUR_WHEEL) -
            medicina_RGB_saturation(COLOUR_WHEEL)) * 5)

    colour.plotting.plot_image(
        np.abs(
            medicina_HSV_control_extended(RGB=COLOUR_WHEEL) -
            medicina_RGB_saturation(COLOUR_WHEEL)) * 5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …